In [1]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [13]:
image_width, image_height = 64, 64
num_classes = 165
images = []
labels = []
class_label_mapping = {}
class_count = 0

In [3]:
dataset_path = r'D:\Project3\tai\EarVN1.0 dataset\Images'

In [6]:
def preprocess_image(image_path):
    img = Image.open(image_path)
    img = img.resize((image_width, image_height))
    img = np.array(img)
    img = img / 255.0 
    return img

In [14]:
for class_name in os.listdir(dataset_path):
    class_folder = os.path.join(dataset_path, class_name)
    if not os.path.isdir(class_folder):
        continue

    # Create a unique integer label for each class
    class_label_mapping[class_name] = class_count
    class_count += 1

    for image_name in os.listdir(class_folder):
        image_path = os.path.join(class_folder, image_name)
        img = preprocess_image(image_path)
        images.append(img)
        labels.append(class_label_mapping[class_name])

In [15]:
images = np.array(images)
labels = np.array(labels)

In [16]:
# One-hot encode the labels
labels = to_categorical(labels, num_classes)

In [17]:
train_images, temp_images, train_labels, temp_labels = train_test_split(images, labels, test_size=0.3, random_state=42)
val_images, test_images, val_labels, test_labels = train_test_split(temp_images, temp_labels, test_size=0.5, random_state=42)

In [18]:
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

train_datagen.fit(train_images)

In [23]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_width, image_height, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [24]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [25]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 31, 31, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 12, 12, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 6, 6, 128)       

In [26]:
model.fit(train_datagen.flow(train_images, train_labels, batch_size=32), epochs=1000, validation_data=(val_images, val_labels))

Epoch 1/1000
622/622 [==============================] - 15s 24ms/step - loss: 5.1004 - accuracy: 0.0087 - val_loss: 5.0891 - val_accuracy: 0.0084
Epoch 2/1000
622/622 [==============================] - 15s 24ms/step - loss: 5.0383 - accuracy: 0.0138 - val_loss: 4.9695 - val_accuracy: 0.0134
Epoch 3/1000
622/622 [==============================] - 14s 23ms/step - loss: 4.9970 - accuracy: 0.0151 - val_loss: 4.9404 - val_accuracy: 0.0155
Epoch 4/1000
622/622 [==============================] - 15s 24ms/step - loss: 4.9738 - accuracy: 0.0173 - val_loss: 4.8980 - val_accuracy: 0.0202
Epoch 5/1000
622/622 [==============================] - 15s 24ms/step - loss: 4.9478 - accuracy: 0.0166 - val_loss: 4.8764 - val_accuracy: 0.0197
Epoch 6/1000
622/622 [==============================] - 15s 24ms/step - loss: 4.9236 - accuracy: 0.0187 - val_loss: 4.8580 - val_accuracy: 0.0235
Epoch 7/1000
622/622 [==============================] - 15s 24ms/step - loss: 4.8926 - accuracy: 0.0201 - val_loss: 4.7795 -

KeyboardInterrupt: 

In [ ]:
loss, accuracy = model.evaluate(test_images, test_labels)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")